In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error


In [ ]:
# Load data
data = pd.read_csv('municipality_bus_utilization.csv', header=None, names=['TIMESTAMP','MUNICIPALITY_ID','USAGE', 'TOTAL_CAPACITY'])

In [ ]:
# Convert TIMESTAMP column to datetime format
data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'], format='%Y-%m-%d %H:%M:%S',errors='coerce')

In [ ]:
municipality_ids_df = pd.DataFrame(municipality_ids, columns=['MUNICIPALITY_ID'])

In [ ]:
# Set TIMESTAMP column as index
data = data.set_index('TIMESTAMP')

# Select only numeric columns
numeric_data = data.select_dtypes(include=[np.number])

# Resample data to hourly frequency and take the max value of each hour
hourly_data = numeric_data.resample('H').max()



In [ ]:
# Resample data to hourly frequency and take the max value of each hour

hourly_data = hourly_data.reindex(data.index, fill_value=np.nan)
data = pd.concat([data, hourly_data], axis=1)


In [ ]:
# Fill missing values with interpolation
hourly_data = hourly_data.interpolate(method='linear')

In [ ]:
# Split data into train and test sets
hourly_data = hourly_data.sort_index()
train_data = hourly_data.loc[:'2017-08-04']

test_data = hourly_data.loc['2017-08-05':]



In [ ]:
# Define evaluation function
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    return mse, mae

In [ ]:
## Train and evaluate baseline model for each municipality_id
baseline_errors = []
for i in range(10):
    train = train_data[train_data['MUNICIPALITY_ID'] == i]['Usage'].values
    test = test_data[test_data['MUNICIPALITY_ID'] == i]['Usage'].values
    predictions = baseline_model(train)
    error = mean_absolute_error(test, predictions)
    baseline_errors.append(error)
    
print('Baseline errors:', baseline_errors)
print('Mean baseline error:', np.mean(baseline_errors))


In [ ]:
# Train and evaluate baseline model for each municipality_id
baseline_errors = []
for i in range(10):
    train = train_data[train_data['MUNICIPALITY_ID'] == i]['USAGE'].values
    test = test_data[test_data['MUNICIPALITY_ID'] == i]['USAGE'].values
    predictions = baseline_model(train)
    mse, mae = evaluate_model(test, predictions)
    baseline_errors.append((mse, mae))
    print(f'Baseline model error for municipality {i}: MSE={mse:.2f}, MAE={mae:.2f}')

In [ ]:
# Train and evaluate SARIMA model for each municipality_id
sarima_errors = []
for i in range(10):
    try:
        train = train_data[train_data['MUNICIPALITY_ID'] == i]['USAGE']
        test = test_data[test_data['MUNICIPALITY_ID'] == i]['USAGE']
        model = SARIMAX(train, order=(1, 0, 1), seasonal_order=(1, 1, 0, 24))
        model_fit = model.fit(disp=False)
        predictions = model_fit.forecast(len(test))
        mse, mae = evaluate_model(test, predictions)
        sarima_errors.append((mse, mae))
        print(f'SARIMA model error for municipality {i}: MSE={mse:.2f}, MAE={mae:.2f}')
    except:
        print(f'Error fitting SARIMA model for municipality {i}.')
        sarima_errors.append((np.nan, np.nan))

In [ ]:
# Print overall error for baseline and SARIMA models
baseline_mse = np.mean([e[0] for e in baseline_errors])
baseline_mae = np.mean([e[1] for e in baseline_errors])
print(f'Overall baseline model error: MSE={baseline_mse:.2f}, MAE={baseline_mae:.2f}')

In [ ]:
sarima_mse = np.nanmean([e[0] for e in sarima_errors])
sarima_mae = np.nanmean([e[1] for e in sarima_errors])
print(f'Overall SARIMA model error: MSE={sarima_mse:.2f}, MAE={sarima_mae:.2